# Document Deduplication with Gaoya

This notebook was based on [Document Deduplication with Similarity Search](https://github.com/pinecone-io/examples/blob/master/deduplication/deduplication_scholarly_articles.ipynb) from Pinecone. The original notebook uses Pinecone vector similarity search to select a set of candidates, which is then further filtered using datasketch MinHash. 
In this notebook we utilize gaoya MinHashIndex to find near duplicates. 

In [2]:
import os
import json
import math
import statistics
import pandas as pd
import gaoya

In this tutorial, we will use the [Deduplication Dataset 2020](https://core.ac.uk/documentation/dataset) that consists of 100,000 scholarly documents.

In [3]:
import requests, os, zipfile

DATA_DIR = "/tmp"
DATA_FILE = f"{DATA_DIR}/deduplication_dataset_2020.zip"
DATA_URL = "https://gaoya.s3.amazonaws.com/deduplication_dataset_2020.zip"


def download_data():
    os.makedirs(DATA_DIR, exist_ok=True)

    if not os.path.exists(DATA_FILE):
        r = requests.get(DATA_URL)  # create HTTP response object
        with open(DATA_FILE, "wb") as f:
            f.write(r.content)
        with zipfile.ZipFile(DATA_FILE, "r") as zip_ref:
            zip_ref.extractall(DATA_DIR)


download_data()

In [4]:
DATA_PATH = os.path.join(DATA_DIR, "deduplication_dataset_2020/Ground_Truth_data.jsonl")

with open(DATA_PATH, encoding="utf8") as json_file:
    data = list(json_file)

Here is a sample of the data.

In [5]:
data_json = [json.loads(json_str) for json_str in data]
df = pd.DataFrame.from_dict(data_json)
df.head(3)

,core_id,doi,original_abstract,original_title,processed_title,processed_abstract,cat,labelled_duplicates
0,11251086,10.1016/j.ajhg.2007.12.013,Unobstructed vision requires a particular refr...,Mutation of solute carrier SLC16A12 associates...,mutation of solute carrier slc16a12 associates...,unobstructed vision refractive lens differenti...,exact_dup,[82332306]
1,11309751,10.1103/PhysRevLett.101.193002,Two-color multiphoton ionization of atomic hel...,Polarization control in two-color above-thresh...,polarization control in two-color above-thresh...,multiphoton ionization helium combining extrem...,exact_dup,[147599753]
2,11311385,10.1016/j.ab.2011.02.013,Lectin’s are proteins capable of recognising a...,Optimisation of the enzyme-linked lectin assay...,optimisation of the enzyme-linked lectin assay...,lectin’s capable recognising oligosaccharide t...,exact_dup,[147603441]


Now let us look at the columns in the dataset that are relevant for our task.

**core_id** - Unique indentifier for each article

**processed_abstract** - This is obtained by applying preprocssing steps like [this](https://spacy.io/usage/processing-pipelines) to the original abstract of the article from the column **original abstract**.

**processed_title** - Same as the abstract but for the title of the article.

**cat** - Every article falls into one of the three possible categories: 'exact_dup','near_dup','non_dup'

**labelled_duplicates** - A list of core_ids of articles that are duplicates of current article

Let's calculate the frequency of duplicates per article. Observe that half of the articles have no duplicates, and only a small fraction of the articles have more than ten duplicates.

In [6]:
lens = df.labelled_duplicates.apply(len)
lens.value_counts()

0     50000
1     36166
2      7620
3      3108
4      1370
5       756
6       441
7       216
8       108
10       66
9        60
11       48
13       28
12       13
Name: labelled_duplicates, dtype: int64

We will make use of the text data to create vectors for every article. We combine the **processed_abstract** and **processed_title** of the article to create a new **combined_text** column. 

In [7]:
# Define a new column for calculating embeddings
df["combined_text"] = df.apply(
    lambda x: str(x.processed_title) + " " + str(x.processed_abstract), axis=1
)

We'll use **MinHashStringindex** from gaoya to minhash everydocument and index them for fast similarity search. Gaoya is implemented in Rust, which is strongly typed compiled language, and types need to be specified at compile time. py-gaoya provides MinHashimplementations for string data with integer ids. 
We convert **core_id** to int and store in a new column with the same name. We do the same for **labeled_duplicates**

In [8]:
df['core_id'] = df['core_id'].astype(int)
df['labelled_duplicates'] = df.labelled_duplicates.apply(lambda x: [int(i) for i in x])

To create MinHash strings needs to be tokenized into words or shingles. Gaoya provides high performance tokenizers, but allows clients to choose any tokenization scheme. 
Here we are using 3-4 character ngrams .

In [24]:
def _analyzer(doc): return doc.split()
minhash_index = gaoya.minhash.MinHashStringIndex(hash_size=32, jaccard_threshold=0.5, num_bands=50, band_size=4, num_hashes=None, analyzer='char', lowercase=False, ngram_range=(3, 4))

In [25]:
minhash_index

MinHashIndex<u32> { threshold = 0.5, num_hashes = 200, bands = 50, rows_per_band = 4, size = 0 } CharShingle((3, Some(4)))

We could insert data using a loop one document at a time. Instead, we call the method `par_bulk_insert_docs` that uses multiple cores to insert data into index in parallel. 

In [26]:
%time minhash_index.par_bulk_insert_docs(list(df['core_id']), list(df['combined_text']))

CPU times: user 27.2 s, sys: 157 ms, total: 27.4 s
Wall time: 2.99 s


In [27]:
df[30000:30005]

,core_id,doi,original_abstract,original_title,processed_title,processed_abstract,cat,labelled_duplicates,combined_text,predicted_duplicates,Correct
30000,33751947,10.1007/s10955-013-0720-1,In this paper we present a novel method to rec...,Bootstrapping Topological Properties and Syste...,bootstrapping topological properties and syste...,reconstruct topological information. topologic...,near_exact_dup,[24767943],bootstrapping topological properties and syste...,[],0
30001,34623027,10.1007/s10260-015-0297-8,Functional data are occurring more and more of...,Multivariate functional outlier detection,multivariate functional outlier detection,occurring analyze them. multivariate observed....,near_exact_dup,[29525269],multivariate functional outlier detection occu...,[],0
30002,34648486,10.1016/j.physletb.2015.11.047,We thank James de Boer and Karsten Riisager fo...,Measurement of the branching ratio for β-delay...,measurement of the branching ratio for β-delay...,james boer karsten riisager helpful cussion an...,near_exact_dup,"[29549700, 81209193]",measurement of the branching ratio for β-delay...,[],0
30003,35079441,10.1016/j.physletb.2014.01.063,We report the first experimental upper bound t...,A first experimental limit on in-matter torsio...,a first experimental limit on in-matter torsio...,torsion neutron parity violation neutron const...,near_exact_dup,"[24976215, 81218577]",a first experimental limit on in-matter torsio...,"[24976215, 81218577]",1
30004,35089352,10.1007/JHEP06(2015)175,We study the η - η ′ mixing up to next-to-next...,"Scrutinizing the η - η ′ mixing, masses and ps...","scrutinizing the η - η ′ mixing, masses and ps...",chiral perturbation phenomenological inputs. o...,near_exact_dup,"[35089431, 60653719, 87082462]","scrutinizing the η - η ′ mixing, masses and ps...","[87082462, 35089431, 60653719]",1


Let's run a query against the index for a one article.

In [28]:
minhash_index.query(df.iloc[30004].combined_text)

[87082462, 35089431, 35089352, 60653719]

Let's compare it with actual labelled_duplicates. Note, that the `query` returns also the id of the query document, where **labelled_duplicates** only contains duplicates, so there will always be an extra id in the result set

In [29]:
df.iloc[30004].labelled_duplicates

[35089431, 60653719, 87082462]

For every article in the dataset we query the index and store the result in the column **predicted_duplicates**

In [30]:
%time df['predicted_duplicates'] = minhash_index.bulk_query(list(df.combined_text.values))

CPU times: user 33.1 s, sys: 228 ms, total: 33.3 s
Wall time: 3.76 s


Remove the id of the query article for every row

In [31]:
_ = df.apply(lambda row: row['predicted_duplicates'].remove(row['core_id']), axis=1)

Let's evaluate the quality of deduplication

In [32]:
df['Correct'] = df.apply(lambda row: set(row['labelled_duplicates']) == set(row['predicted_duplicates']), axis=1).astype(int)

In [33]:
prediction_summary = { 'Correct' : df['Correct'].sum(), 'Incorrect' : df.shape[0] - df['Correct'].sum() }
prediction_summary['Accuracy'] = round(prediction_summary['Correct'] / df.shape[0], 4)

In [34]:
prediction_summary

{'Correct': 93807, 'Incorrect': 6193, 'Accuracy': 0.9381}

We also calculate recall and precision of the deduplication 

In [35]:
def _recall(row):
    labelled_dups = set(row['labelled_duplicates'])
    if len(labelled_dups) == 0:
        return 1
    dups = set(row['predicted_duplicates'])
    return len(dups & labelled_dups) / len(labelled_dups)
recalls = df.apply(lambda row: _recall(row), axis=1)

In [36]:
recalls.mean()

0.9623026827616825

In [37]:
def _precision(row):
    labelled_dups = set(row['labelled_duplicates'])
    dups = set(row['predicted_duplicates'])    
    if len(dups) == 0:
        return 0

    return len(dups & labelled_dups) / len(dups)
precisions = df.apply(lambda row: _precision(row), axis=1)

In [38]:
precisions.mean()

0.4642050719943308

## Summary

In this notebook we demonstrate how to perform a deduplication task of over 100,000 articles using Gaoya. High performance MinHash algorithm implemented in  Rust  allows deduplicate 100K dataset in just a few seconds
